In [109]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense,LSTM,Flatten
from keras.layers import Dropout
from keras.models import Sequential

In [110]:
df=pd.read_csv("train_data.csv")

In [111]:
df.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"


In [112]:
df=df.iloc[:,1:2]

In [113]:
df

,Open
0,325.25
1,331.27
2,329.83
3,328.34
4,322.04
...,...
1253,790.90
1254,790.68
1255,793.70
1256,783.33


In [114]:
df.shape

(1258, 1)

In [115]:
scaler=MinMaxScaler()
df=scaler.fit_transform(df)

In [116]:
# creating a data structure with 60 time steps and 1 output
x_train=[]
y_train=[]
for i in range(60,len(df)):
    x_train.append(df[i-60:i,0])
    y_train.append(df[i,0])

In [117]:
x_train,y_train=np.array(x_train),np.array(y_train)

In [118]:
# Reshaping
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))

In [119]:
x_train.shape

(1198, 60, 1)

In [120]:
# building an RNN
model=Sequential()
# first lstm layer and adding some dropout regularisation
model.add(LSTM(units=50,return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(Dropout(.2))
# second lstm layer and adding some dropout
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(.2))
# third lstm layer and adding some dropout
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(.2))
# fourth lstm layer and adding some dropout
model.add(LSTM(units=50))
model.add(Dropout(.2))
# adding outputlayer
model.add(Dense(units=1))

In [121]:
model.compile(optimizer="adam",loss="mean_squared_error")

In [ ]:
model.fit(x_train,y_train,epochs=100,batch_size=32)

Epoch 1/100
38/38 [==============================] - 1s 19ms/step - loss: 0.0946
Epoch 2/100
38/38 [==============================] - 1s 19ms/step - loss: 0.0079
Epoch 3/100
38/38 [==============================] - 1s 20ms/step - loss: 0.0068
Epoch 4/100
38/38 [==============================] - 1s 19ms/step - loss: 0.0072
Epoch 5/100
38/38 [==============================] - 1s 20ms/step - loss: 0.0077
Epoch 6/100
38/38 [==============================] - 1s 20ms/step - loss: 0.0085
Epoch 7/100
38/38 [==============================] - 1s 20ms/step - loss: 0.0071
Epoch 8/100
38/38 [==============================] - 1s 21ms/step - loss: 0.0081
Epoch 9/100
38/38 [==============================] - 1s 20ms/step - loss: 0.0097
Epoch 10/100
38/38 [==============================] - 1s 19ms/step - loss: 0.0072
Epoch 11/100
38/38 [==============================] - 1s 19ms/step - loss: 0.0065
Epoch 12/100
38/38 [==============================] - 1s 19ms/step - loss: 0.0068
Epoch 13/100
38/38 [=====

In [ ]:
# making the prediction and visualizing the results
dataset_test=pd.read_csv("test_data.csv")

In [ ]:
dataset_test.head()

In [ ]:
dataset_train=pd.read_csv("train_data.csv")

In [ ]:
total_dataset=pd.concat([dataset_train["Open"],dataset_test["Open"]],axis=0)

In [ ]:
total_dataset.shape

In [ ]:
inputs=total_dataset[len(total_dataset)-len(dataset_test)-60:].values

In [ ]:
inputs.shape

In [ ]:
inputs=inputs.reshape(-1,1)

In [ ]:
inputs.shape

In [ ]:
inputs=scaler.transform(inputs)

In [ ]:
x_test=[]
for i in range(60,80):
    x_test.append(inputs[i-60:i,0])

In [ ]:
x_test=np.array(x_test)

In [ ]:
x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))

In [ ]:
x_test.shape

In [ ]:
predict=model.predict(x_test)
predicted_price=scaler.inverse_transform(predict)

In [ ]:
predicted_price

In [ ]:
real_price=dataset_test.iloc[:,1:2].values

In [ ]:
# visualising the result
plt.plot(real_price,label="real google price")
plt.plot(predicted_price,label="predicted price")
plt.title("google stock price prediction")
plt.xlabel("time")
plt.ylabel("google stockprice")
plt.legend()
plt.show()